# <font face="Verdana" size=6 color='#6495ED'>  KALMAN FILTERS - Hands On

<font face="Verdana" size=3 color='#40E0D0'> Profs. Larissa Driemeier, Arturo Forner-Cordero and Thiago Martins

<center><img src='https://drive.google.com/uc?export=view&id=1fV0ruX4TCznLnNw-3ZV390p2JyU_-llu' width="600"></center>


## INTRODUCTION: Presentation of the problem

Visual information is the most important type of information perceived, processed and interpreted by the human brain. As we know, computers don't have *eyes*, so, how can they still find patterns in an image?

Digital image processing is the use of a computer to perform some operations on digital images by means of an algorithm, in order to get enhanced image either to extract some useful information.

Before we jump into image processing world, let's define what exactly constitutes an image. An image is nothing more than a two-dimensional matrix (3-D in the case of color images) defined by $f(x,y)$ where $x$ and $y$ represent the dimensions (width and height, respectively) based on the number of pixels. Each pixel is an image point. The value of $f(x,y) $ describes a specific shade, opacity or color of the pixel. It is usually represented as:

* Grayscale - $f(x,y)$ is a single number stored as an 8-bit integer with value between 0 to 255, representing the brightness of the pixel (0:  black; 255: white).
* RGB - to represent color images, $f(x,y)$ is made up of 3 integers representing the components of red (R), green (G) and blue (B) for each pixel. Thus, “value” of the pixel becomes a vector of three numbers. For example, let's take the YELLOW color. It is formed by combining these three colors where the RED (intensity) value is 255, the BLUE is 255 and the GREEN is 0. Often, the three different components are stored as three separate *grayscale* images known as color planes, which need to be recombined during display or processing.
* RGBA - is an extension of RGB with an added alpha variable, varying from 0 to 1, which represents the opacity of the image ( 0:fully transparent; 1:fully opaque).

<center><img src='https://drive.google.com/uc?export=view&id=1vA-pDXxBOBbCjjRXibLiOgdleBIuXFB9' width="500"></center>

* HSV - for hue, saturation, value is an alternative representation of the RGB color model, designed in the 1970s.  HSV is closer to how humans perceive color and it usually appears as a cone or cylinder with these three components: *Hue* is the color portion of the model, expressed as a number from 0 to 360 degrees (Red: 0 - 60 degrees, Yellow: 61 - 120 degrees; Green: 121 - 180 degrees, Cyan: 181 - 240 degrees, Blue 241 - 300 degrees, Magenta: 301 - 360 degrees); *Saturation* describes the amount of gray in a particular color, from 0 to 100% (saturation toward zero introduces more gray and produces a faded effect);  *Value* (or Brightness) acts with saturation and defines the brightness or intensity of the color, from 0 to 100%, where 0 is completely black.

<center><img src='https://drive.google.com/uc?export=view&id=1gJziCnLTJcOfk8aLkO2XRCSdDkGzQFGa' width="250"></center>

* CIELAB - *CIE* refers to *Commission internationale de l'éclairage* (International Commission on Illumination), an independent, non-profit organization devoted to worldwide cooperation and the exchange of information on all matters relating to the science and art of light and lighting, colour and vision, photobiology and image technology. *Lab* is composed of a "L" representing brightness, ranging from 0-100 - the larger the value of L, the brighter; *a* stands for red and green: the value from negative to positive is the evolution from green to red; *b* stands for blue and yellow:the evolution from negative to positive is the evolution from blue to yellow.

<center><img src='https://drive.google.com/uc?export=view&id=1Jg0AjJ2MP5uoYiw89oDqX49BkCZ9aXiH' width="400"></center>

In this way, dealing with the values of $f(x,y)$, computers can process, understand, find patterns, etc... in an image.




In [ ]:
!pip install ffmpeg-python

from IPython.display import HTML, Javascript, display
from base64 import b64encode

from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg

import imutils

import cv2, sys, numpy, os
from time import sleep
from google.colab.patches import cv2_imshow

## Your work today

You will use computer vision and Kalman Filter to track a ball down a ramp, as shown in the diagram below.

## Mount Google Drive to store captured videos

The first few lines of code allow you to mount your Google Drive to store captured videos and images.

Initially, you will be asked to log into your Google account and confirm permissions.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Now you will be able to see your Google Drive folders directly from Google Colab.

<img src='https://drive.google.com/uc?export=view&id=1CLTPpD7L5gEzFnmYnKuGAoBQiA2lx8dy' width="200">

Create the following folders on your drive:
* My Drive > Colab Notebooks > KALMAN_2023 > Video Datasets
* My Drive > Colab Notebooks > KALMAN_2023 > Image Datasets

In [ ]:
import pathlib
import os
import os.path

# Folders to store videos and images
video_path = '/content/drive/MyDrive/Colab Notebooks/KALMAN_2024/Video Datasets'
image_path = '/content/drive/MyDrive/Colab Notebooks/KALMAN_2024/Image Datasets'
os.chdir(".")
print("current dir is: %s" % (os.getcwd()))

if os.path.isdir(video_path):
    print("Folder exists.")
else:
    print("Folder does not exist.")
    os.mkdir(video_path)
    print("Folder created!")

if os.path.isdir(image_path):
    print("Folder exists.")
else:
    print("Folder does not exist.")
    os.mkdir(image_path)
    print("Folder created!")

In [ ]:
# Subfolders of 'Image Datasets' used to store the images:
sub_data = 'Result_NoFilter'
sub_data2 = 'Result_Filter'
# Folders to store images and videos
path_nofilter = os.path.join(image_path, sub_data)
path_filter = os.path.join(image_path, sub_data2)
path = pathlib.Path(path_nofilter)
path.mkdir(parents=True, exist_ok=True)
path = pathlib.Path(path_filter)
path.mkdir(parents=True, exist_ok=True)

##  <font face="Verdana" size=6 color='#6495ED'> Part I: create videos

With the ball we provided, you must record two videos:

<center><img src='https://drive.google.com/uc?export=view&id=17eufK1U-VRFSsaBS6IGBY2lAZDPmulYM' width="900"></center>



1. video of the ball going down the ramp, according to the image below. Note that the dimensions are approximate and you do not need them to implement your filter. They serve to guide you in your filming. Your challenge will be to make the computer understand what happened to the ball when it disappears behind the red box! In fact, a great challenge for autonomous vehicles: *if something has disappeared for a few moments from the cameras' field of view, it doesn't mean that it doesn't really exist anymore*.  For visual tracking, the constant velocity motion model is the most widely used process model as it can describe e.g. pedestrian movements very well.

<center><img src='https://drive.google.com/uc?export=view&id=1SUvhDXYuxLATuXirtH9VERC92cXBdZ6u' width="800"></center>

2. video of the ball bouncing on the ground (or table) 2 or 3 times or transfer the ball from one hand to another 2 or 3 times.

<center><img src='https://drive.google.com/uc?export=view&id=1CRwNHOeo6jgh5XGzDSLqTE3k96KedqDy' width="800"></center>

You should create the videos using a cell phone, taking into account that the camera shakes during filming and cut the beginning and ending parts of the video that are irrelevant and will confuse the filter. Film frames are illustrated in Fig.~\ref{fig:videoParts}. When the sensor misses the ball, the Kalman filter, through the prediction step, must be able to estimate its path.

Use the video available on Moodle as a guide.

__Attention__: the video `Ramp_YellowBall.mp4` is only for guide you. You must use your own video!

The function below returns the number of frames for the video specified as input. In OpenCV, the first step towards reading a video file is to create a `VideoCapture` object. Once the `VideoCapture` object has been created, you can use the `read()` method to *read* frames from the video stream. It returns two values: a Boolean value indicating whether the read operation was successful (`grabbed` in the function) and the actual image data in the form of a NumPy array (`frame`). Moreover, the function `shape` returns the dimensions of a frame (in pixels) and number of channels.

In [ ]:
def count_frames_manual(video):
	# initializes the counter for the number of frames
	total = 0
	# loop over video frames
	while True:
		# get the current frame
		grabbed, frame = video.read()

		# check if we reached the end of the video
		if not grabbed:
			break
		# increments the total number of frames
		total += 1
		height, width, channels = frame.shape
	# return the total number of frames in the video file, height, width and #channels of each frame
	return total,height, width, channels

###  <font face="Verdana" size=5 color='#40E0D0'>  Part I: delivery 1#3

* Upload the videos* in MP4 format.
* Write and submit a small report* for each video following the example below:
 * Insert a table with: frame dimension, frames per second, total frames and total time.
 * Choose three frames and insert the information about the time ($t$).

<small>*For both experiments<snall>

 <center><img src='https://drive.google.com/uc?export=view&id=1V3NVcu0QFN_GkDK6vjUllhtbcFvSl9aP' width="800"></center>

##  <font face="Verdana" size=6 color='#6495ED'> Part II: Ball detection

Now let's detect the ball by image processing. __Repeat for both videos you recorded.__

### Upload the video

Upload the video. It will be used for tracking the ball.

In [ ]:
video_name = 'Ramp_YellowBall.mp4'
video_file = os.path.join(video_path , video_name)

In [ ]:
if os.path.exists(video_file):
  print('Video available on drive.')
  video = cv2.VideoCapture(video_file)
  fps = video.get(cv2.CAP_PROP_FPS)
  total,height, width, channels = count_frames_manual(video)
  print ("Frames per second   : {:10.4f}".format(fps))
  print ("Total frames        : {:d}".format(total))
  print ("Total time, in  [s] : {:10.4f}".format(total/fps))
  print ("Dimension of a frame: {:d} x {:d}".format(height, width))
  video.release()
  cv2.destroyAllWindows()
else:
  print('Video is not available.')

In [ ]:
mp4 = open(video_file,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

### Ball tracking creation



In [ ]:
# Detection margins

# Black
HSV_lower_black = np.array([0,0,0], dtype=np.uint8)
HSV_upper_black = np.array([170,150,50], dtype=np.uint8)

# Red
HSV_lower_red = np.array([161, 155, 84])
HSV_upper_red = np.array([179, 255, 255])

# Blue
HSV_lower_blue = np.array([94, 80, 2])
HSV_upper_blue = np.array([126, 255, 255])

# Green
HSV_lower_green = np.array([25, 52, 72])
HSV_upper_green = np.array([102, 255, 255])

# Yellow
HSV_lower_yellow = np.array([22, 93, 0], dtype="uint8")
HSV_upper_yellow = np.array([45, 255, 255], dtype="uint8")

# Any color except white
HSV_lower = np.array([0, 42, 0])
HSV_upper = np.array([179, 255, 255])

In [ ]:
def detect_ball(frame, HSV_lower, HSV_upper, Flag_Measurement=False):
    """
     Generates a circle around the ball, identifying its position in the frame
     Args:
         frame (np.ndarray[float])        : Frame image array
         HSV_lower (np.ndarray[int])      : Array with the lower range with HSV data (Hue, Saturation, Value)
                                            to create a mask according to the color of the ball.
         HSV_upper (np.ndarray[int])      : Array with the upper range with HSV data  (Hue, Saturation, Value)
                                            to create a mask according to the color of the ball.
         Flag_Measurement (bool, optional): Indicates whether the ball was detected. Default is False.
     Returns:
         Tuple[float, float, float, bool] : Returns the position of the center of the ball (center[0], center[1]),
                                            radius of the ball (radius) and the flag (Flag_Measurement).
    """
    x, y, radius = -1, -1, -1
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#
    mask = cv2.inRange(hsv_frame, HSV_lower, HSV_upper)
    # uncomment if you cannot understand what the command `cv2.inRange` does
    # Be warned that the image will be shown for each frame of the video.
    # cv2_imshow(mask)

    blur = cv2.medianBlur(mask, 11)
    # uncomment if you cannot understand what the command `cv2.medianBlur` does
    # Be warned that the image will be shown for each frame of the video.
    #cv2_imshow(blur)

    circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1, 100, param1=500, param2=10, minRadius=10, maxRadius=100)
    # try to understand the minimum necessary about `cv2.HoughCircles` parameters with the OpenCV documentation
    center = (-1,-1)
    radius = 0.
#
    if circles is not None:
        Flag_Measurement = True
        center = (circles[0][0][0],circles[0][0][1])
        radius = circles[0][0][2]
    return center[0], center[1], radius, Flag_Measurement

In [ ]:
if not os.path.isdir(path_nofilter):
  os.mkdir(path_nofilter)

# Video
webcam = cv2.VideoCapture(video_file)
(_, frame) = webcam.read()
cv2_imshow(frame)

### Final video creation and playback

After you run the code below, all frames of the film with the ball tracking are saved as `.png` in the folder,

My Drive > Colab Notebooks > KALMAN_2023 > Image Datasets > Result_NoFilter

The command used for this purpose is
```
cv2.imwrite('% s/% s.png' % (path, count), frame)
```

That is, each frame of the movie with the the ball contour and its center added as superscript is saved in the folder defined as `path`.

In [ ]:
N = total

count = 0
while count < N-1:
    if not webcam.isOpened():
        print('Unable to open the video.')
        sleep(5)
        pass
    (_, frame) = webcam.read()
#
    # Detect Ball in the frame
    [x,y,radius, Flag] = detect_ball(frame, HSV_lower_yellow, HSV_upper_yellow)
    z = [x,y]
    #
    if radius > 10:
       #
       cv2.circle(frame, (int(x), int(y)), int(radius), (102, 102, 0), 2)
       #
       cv2.circle(frame, (int(z[0]), int(z[1])), 3, (153, 153, 0), -1)

       if (count != 0):
         cv2.putText(frame, 'Track', (int(z[0]), int(z[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (16,128,4), 2)
         cv2.putText(frame, 'Center coordinates x: '+str(int(z[0]))+' y: '+str(int(z[1])), (120,460), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (128,33,4), 2)

    cv2.imwrite('% s/% s.png' % (path_nofilter, count), frame)
    count += 1
webcam.release()
cv2.destroyAllWindows()

Now, you will turn those images into a video and save it with `BallTrackingResult_NoFilter.mp4`.



In [ ]:
images = [img for img in os.listdir(path_nofilter) if img.endswith(".png")]

frame = cv2.imread(os.path.join(path_nofilter, images[0]))
height, width, layers = frame.shape

video_name_result = 'BallTrackingResult_NoFilter.mp4'
video_path_result = os.path.join(video_path , video_name_result)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_path_result, fourcc, fps/3, (width,height))

for image in images:
    frame = cv2.imread(os.path.join(path_nofilter, image))
    video.write(frame)

video.release()
print('Video generated successfully.')

Watch the video in the drive.

###  <font face="Verdana" size=5 color='#40E0D0'> Part II: delivery 2#3

* The objective is to evaluate your understanding about the ball detection code. Summarize the method used in the `detect_ball` function. For example, explain the commands `cv2.inRange`, `cv2.medianBlur`and `cv2.HoughCircles`. What is the output of each function? Then understand how `detect_ball` function is called. Flowchart the ball detection algorithm.
* Submit both videos with the tracking (via computational vision) you just did.
* What are your conclusions about the videos? __Attention: do not ask the professor's opinion, summarize the whole group discussion and highlight the most important points .__

Now, have fun with Kalman...

##  <font face="Verdana" size=6 color='#6495ED'> Part III: Kalman Filter

The summary of the Kalman filter formulation presented in the Theory class Notebook is repeated here.

$$
\begin{aligned}
\text{Predict Step}\\
\mathbf{\bar x} &= \mathbf{F x} + \mathbf{B u} \\
\mathbf{\bar P} &= \mathbf{FP{F}}^\mathsf T + \mathbf Q \\
\\
\text{Update Step}\\
\textbf{S} &= \mathbf{H\bar PH}^\mathsf T + \mathbf R \\
\mathbf K &= \mathbf{\bar PH}^\mathsf T \mathbf{S}^{-1} \\
\textbf{y} &= \mathbf z - \mathbf{H \bar x} \\
\hat{\mathbf x} &=\mathbf{\bar x} +\mathbf{K\textbf{y}} \\
\hat{\mathbf P}  &= (\mathbf I - \mathbf{KH})\mathbf{\bar P}(\mathbf I - \mathbf{KH})^\mathsf T + \mathbf{KRK}^\mathsf T
\end{aligned}
$$

There is a simplification for the covariance matrix defined in the theoretical class:
$$\hat{\mathbf P}  = (\mathbf{I}-\mathbf{KH})\mathbf{\bar P}$$




### System definition

Now your goal is to get the formulation for $\mathbf F_t, \mathbf Q_t, \mathbf H$ and $\mathbf R_t$.

In this problem, the system state vector given by: \begin{equation*}
  \mathbf x_t = \begin{bmatrix} x \\ y \\ v_x \\ v_y \end{bmatrix}
 \end{equation*}

where $x, y$ are the coordinates of the ball's position, $v_x, v_y$ are the respective velocity components.

Attention, __acceleration is not a state quantity__. However, the assumption of perfect constant speed is unrealistic, especially for real-world applications. A more sophisticated model of acceleration can be created, based on the angle of the ramp, rolling friction and the aerodynamic drag of the ball. The difficulty with this approach is that, in addition to demanding the characterization of model parameters, it produces an acceleration in the global reference. The transformation from the global reference to the camera's reference is unknown. __Therefore, we will use the constant velocity model, which is the most used in these cases.__ A relaxation to this strong assumption is allowed by introducing a piecewise constant *white acceleration*. Therefore, a  recurrence law of evolution of the state vector can be written as follows:

\begin{equation}
 \mathbf x_{t+1} = \mathbf F\mathbf x_{t}+ \mathbf B \mathbf u_{t+1} + \mathbf w_{t+1}
\end{equation}
where ${\mathbf{w}_t \sim N(\mathbf 0,\mathbf Q_t)}$ is a Gaussian random vector with zero mean and covariance matrix $\mathbf Q_t$, which sums to the deterministic model. Disregarding $\mathbf B \mathbf u_{t}$ given that the proposed problem does not include control, one can make explicit eq. (1),
\begin{align}
  x_{t+1} &= x_{t}+ \delta t \, \dot{x}_{t}+\frac{\delta t^2}{2}\omega_{x,t}\\
  y_{t+1} &= y_{t}+ \delta t \, \dot{y}_{t}+\frac{\delta t^2}{2}\omega_{y,t}\\
  \dot{x}_{t+1} &= \dot{x}_{t}+ \delta t \, \omega_{x,t}\\
  \dot{y}_{t+1} &= \dot{y}_{t}+ \delta t \, \omega_{y,t}\\
\end{align}
where the time step $\delta t$ is defined as the time between measurements, $\omega_{x,t},\omega_{y,t}$ are the process noise corresponding to $x, y$, respectively. These are basically zero-mean Gaussian white noise. In matrix form, we have:
$$
     \begin{bmatrix}
         x_{t+1} \\
         y_{t+1} \\
         \dot{x}_{t+1} \\
         \dot{y}_{t+1}  
     \end{bmatrix}
     =
     \begin{bmatrix}
         1 & 0 & \delta t & 0 \\
         0 & 1 & 0 & \delta t \\
         0 & 0 & 1 & 0 \\
         0 & 0 & 0 & 1  
     \end{bmatrix}
      \times
     \begin{bmatrix}
         x_{t} \\
         y_{t} \\
         \dot{x}_{t} \\
         \dot{y}_{t}  
     \end{bmatrix}+ \begin{bmatrix}
         \frac{\delta t^2}{2}\omega_{x,t} \\
         \frac{\delta t^2}{2}\omega_{y,t} \\
         \delta t\,\omega_{x,t} \\
         \delta t \, \omega_{y,t}  
     \end{bmatrix}
  $$
$$
\mathbf Q_{t} =
     \begin{bmatrix}
          \frac{\delta t^4}{4}\sigma^2_{\omega_x} & 0 & \frac{\delta t^3}{2}\sigma^2_{\omega_x} & 0 \\
         0 &  \frac{\delta t^4}{4}\sigma^2_{\omega_y} & 0 & \frac{\delta t^3}{2}\sigma^2_{\omega_y} \\
         \frac{\delta t^3}{2}\sigma^2_{\omega_x} & 0 & \delta t^2 \sigma^2_{\omega_x}& 0 \\
         0 & \frac{\delta t^3}{2}\sigma^2_{\omega_y} & 0 & \delta t^2 \sigma^2_{\omega_y}  
     \end{bmatrix}
$$
To derive the equation of $\mathbf Q_{t}$, two ideas are important:
* $\mathbb{E}\left[ \omega_{x,t} \omega_{x,t}\right] = \sigma^2_{\omega_x}$ and $\mathbb{E}\left[ \omega_{ y,t} \omega_{y,t}\right] = \sigma^2_{\omega_y}$, where $\sigma^2_{\omega_x},\sigma^2_{\omega_y}$ are the variances.
* $\mathbb{E}\left[ \omega_{x,t} \omega_{y,t}\right] = 0$ because there is no correlation between the $x$ axis and the $y$ axis.

Measurements are represented by the vector:
\begin{equation*}
 z_t = \begin{bmatrix} c_x \\ c_y \end{bmatrix}
\end{equation*}
where $c_x,c_y$ are the coordinates of the center of the sphere (see what the `detect_ball` function returns) and will be considered our *measurements* here. These measurements show independent Gaussian random noise with zero-mean and covariances $r_x^2$ and $r_y^2$, for $x$ and $y$ respectively.

Write the system measurement law in the following form:
\begin{equation}
 \mathbf z_{t+1} = \mathbf H \mathbf {\bar x}_t + \mathbf v_{t}
\end{equation}
where ${\mathbf{v}_t \sim N(\mathbf 0,\mathbf R_t)}$ is a Gaussian random vector with zero mean and covariance matrix $\mathbf R_t$.
$$
\mathbf R_{t} =\mathbb{E}\left[ \begin{bmatrix}
         v_{x,t} \\
         v_{y,t}
     \end{bmatrix} \begin{bmatrix}
         v_{x,t} &  v_{y,t}
     \end{bmatrix} \right]
  $$
such that:
\begin{equation}
 \mathbf R_t = \begin{bmatrix} r_x^2 & 0 \\ 0 & r_y^2  \end{bmatrix}
\end{equation}
if:
* $\mathbb{E}\left[ v_{x,t} v_{x,t}\right] = r^2_x$ and $\mathbb{E}\left[ v_{y,t} vp_{y,t}\right] = r^2_y$, where $r^2_x,r^2_y$ are variances.
* $\mathbb{E}\left[p_{x,t} p_{y,t}\right] = 0$ because there is no correlation between the measurements of the $x$ and $y$ axes.

It is also important to note that, in general, variances $\sigma^2_{\omega_x} = \sigma^2_{\omega_y}$ and $r^2_x=r^2_y$ are adopted for simplification. This is what we will do here. However, it may be important to adjust them accordingly. __Always think carefully about the parameters you will use in a project.__


### Kalman Filter Algorithm

Here you will generate your "Kalman Filter" class. The functions are:
* `Predict`, which will predict with your model and process;
* `Update`, which will make the correction, from the camera measurement;
* `ellipseP`, will draw the ellipse (with 2 standard deviations) referring to the covariance `P`. Substitute de circle with the ellipse while you're tracking the ball.

See that, in the video, the ellipse with 2 standard deviations is drawn (in red) over the ball.



<center><img src='https://drive.google.com/uc?export=view&id=17dTAAiHUkPbMRMVU3EBjOA1Cv8ayHjWv' width="400"></center>


In [ ]:
class KalmanFilter(object):
    def __init__(self, dt, u_x,u_y, var_w, varz_x, varz_y, initial_var_pos, initial_var_vel):
        """
          :param dt: time step (time for 1 cycle)
          :param u_x: acceleration in the x direction
          :param u_y: acceleration in the y direction
          :param var_w: process noise variance
          :param varz_x: measurement variance in the x direction
          :param varz_y: measurement variance in the y direction
        """
        # Period
        self.dt = dt
        # Control variables
        self.u = np.array([[u_x],[u_y]])
        # Initial state
        self.x = np.array([[500], [300], [0], [0]])
        # State Transition Matrix F
        self.F = np.array()
        # Control Input Matrix B
        self.B = np.array()
        # Measurement Mapping Matrix
        self.H = np.array()
        # Process Noise Covariance Q
        self.Q = np.array()
        # Measurement Noise Covariance R
        self.R = np.array()
        # Initial Covariance Matrix P
        self.P = np.array()

    def initial(self,cx,cy):
        self.x =[cx,cy,0,0]
        return

    def predict(self):
        self.x =
        self.P =
        return self.x[0:2]

    def update(self, cx,cy):
        z = np.array([[cx],
                     [cy]])
        S =
        K =
        self.x =
        self.P =
        return self.x[0:2]

    def ellipseP(self,nstd = 2):
      #nstd is the #standard deviations
      cov = [[self.P[0,0],self.P[0,1]],[self.P[1,0],self.P[1,1]]]
      vals, vecs = np.linalg.eigh(cov)
      theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    # width, height are diameters, not radius
      width, height = 2 * nstd * np.sqrt(vals)
      return width, height, theta

In [ ]:
# @title Response
class KalmanFilter(object):
    def __init__(self, dt, u_x,u_y, var_w, varz_x, varz_y, initial_var_pos, initial_var_vel):
        """
          :param dt: time step (time for 1 cycle)
          :param u_x: acceleration in the x direction
          :param u_y: acceleration in the y direction
          :param var_w: process noise variance
          :param varz_x: measurement variance in the x direction
          :param varz_y: measurement variance in the y direction
        """
        # Period
        self.dt = dt
        # Control variables
        self.u = np.array([[u_x],[u_y]])
        # Initial state
        self.x = np.array([[500], [300], [0], [0]])
        # State Transition Matrix F
        self.F = np.array([[1, 0, self.dt, 0],
                            [0, 1, 0, self.dt],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])
        # Control Input Matrix B
        self.B = np.array([[(self.dt**2)/2, 0],
                            [0, (self.dt**2)/2],
                            [self.dt,0],
                            [0,self.dt]])
        # Measurement Mapping Matrix
        self.H = np.array([[1, 0, 0, 0],
                            [0, 1, 0, 0]])
        # Process Noise Covariance Q
        self.Q = np.array([[(self.dt**4)/4, 0, (self.dt**3)/2, 0],
                            [0, (self.dt**4)/4, 0, (self.dt**3)/2],
                            [(self.dt**3)/2, 0, self.dt**2, 0],
                            [0, (self.dt**3)/2, 0, self.dt**2]]) * var_w
        # Measurement Noise Covariance R
        self.R = np.array([[varz_x,0],
                           [0, varz_y]])
        # Initial Covariance Matrix P
        self.P = np.array([[initial_var_pos, 0., 0., 0.],
                           [0., initial_var_pos, 0., 0.],
                           [0., 0., initial_var_vel, 0.],
                           [0., 0., 0., initial_var_vel]])

    def predict(self):
        self.x = self.F.dot(self.x) #+ self.B.dot(self.u)
        self.P = self.F.dot(self.P).dot(self.F.T) + self.Q
        return self.x[0:2]

    def update(self, cx,cy):
        z = np.array([[cx],
                     [cy]])
        S = self.H.dot(self.P).dot(self.H.T) + self.R
        K = np.array(self.P.dot(self.H.T).dot(np.linalg.inv(S)))
        self.x = self.x + K.dot(z - self.H.dot(self.x))
        A = np.eye(self.H.shape[1]) - K.dot(self.H)
        self.P = A.dot(self.P).dot(A.T)+K.dot(self.R).dot(K.T)
        return self.x[0:2]

    def ellipseP(self,nstd = 2):
      #nstd is the #standard deviations
      cov = [[self.P[0,0],self.P[0,1]],[self.P[1,0],self.P[1,1]]]
      vals, vecs = np.linalg.eigh(cov)
      theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    # width, height are diameters, not radius
      width, height = 2 * nstd * np.sqrt(vals)
      return width, height, theta

### Parameters
Here, the parameters are already defined for you. Find an explanation for their values.

If you do not agree, you can change. But, remember that you will be alone, sailing through uncharted waters. Ok, you will be engineers and it's good to start having these feelings...

<center><img src='https://drive.google.com/uc?export=view&id=1Rjoq7CKl6EE46Ec0xef13ko5n7WGilMX' width="250"></center>

__Some tips__

The dimension of a frame is 480 (H) x 848 (W) and the ramp that is $\approx 1m$ wide and occupies $\approx 540$ pixels in the picture.

The tennis ball diameter is $\approx 0.067m$, which means $\approx 36$ pixels. The *measurement error* is assumed to be around$1/3-1/4$ of the radius, that is, $\approx \pm 5$ pixels.

For process variance, remember that space unity is pixel, acceleration $g = 9.8 m/s^2$ should be $pixels/s^2$. Moreover, remember that $\sigma^2_{a}=\mathbb{E}[\omega\omega^T]$ and $\omega$ is somewhere from $\frac{1}{6} g$ to $g$.

 __REMEMBER__ Prior to initiating the Kalman filter, it is essential to obtain measurements from the sensor. This step is considered good practice as it allows for the updating of the initial state estimate, enhancing the accuracy and reliability of the filter's performance.

<center><img src='https://drive.google.com/uc?export=view&id=1lpo5Z9MNK9mwMrmVxkslbbEC9sK8shrz' width="1400"></center>


In [ ]:
## KALMAN FILTER
# Period(delta t)
dt = 1/fps
# Control input (u)
u_x = 0.
u_y = 0.
# Process variance (Q)
var_w =
# Measurement variance (R)
varz_x =
varz_y =
# initial variance (P)
initial_var_vel = 400.
initial_var_pos = 400.

KF = KalmanFilter(dt, u_x, u_y, var_w, varz_x, varz_y, initial_var_pos, initial_var_vel)

During the implementation, rerun from this point to test your code. Don't forget to put the name of your video in the variable `video_name_result`.

In [ ]:
# Folders to store videos and images
video_path = '/content/drive/MyDrive/Colab Notebooks/KALMAN_2024/Video Datasets'
image_path = '/content/drive/MyDrive/Colab Notebooks/KALMAN_2024/Image Datasets'
# Subfolders of 'Image Datasets' used to store the images:

sub_data2 = 'Result_Filter'
video_name_result = 'BallTrackingResult_Filter.mp4'
# Folders to store images and videos
path_filter = os.path.join(image_path, sub_data2)
imgs = os.listdir(path_filter)
for img in imgs:
    os.remove(f'{path_filter}/{img}')
try:
    os.remove(f'{video_path}/{video_name_result}')
except:
    print("Not such file", video_name_result)


In [ ]:
# Video
webcam = cv2.VideoCapture(video_file)
(_, frame) = webcam.read()
cv2_imshow(frame)

In [ ]:
# @title Response
N = total

count = 0
while count < N-1:
    if not webcam.isOpened():
        print('Unable to load the video file.')
        sleep(5)
        pass
    (_, frame) = webcam.read()
    #detect_ball(frame)
    [x,y,radius, Flag] = detect_ball(frame, HSV_lower_yellow, HSV_upper_yellow)
    z = [x,y]

    if (count != 0 ):
###############
## YOUR CODE ##
###############

    count += 1
webcam.release()
cv2.destroyAllWindows()

In [ ]:
images = [img for img in os.listdir(path_filter) if img.endswith(".png")]

frame = cv2.imread(os.path.join(path_filter, images[0]))
height, width, layers = frame.shape

video_name_result = 'BallTrackingResult_Filter.mp4'
video_path_result = os.path.join(video_path , video_name_result)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_path_result, fourcc, fps/3, (width,height))

for image in images:
    frame = cv2.imread(os.path.join(path_filter, image))
    video.write(frame)

video.release()
print("Video generated successfully.")

### Result

The code below is not working for the video `BallTrackingResult_Filter.mp4`.

<center><img src='https://drive.google.com/uc?export=view&id=1Rjoq7CKl6EE46Ec0xef13ko5n7WGilMX' width="250"></center>



In [ ]:
mp4 = open(video_path_result,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

###  <font face="Verdana" size=5 color='#40E0D0'> Part III: delivery 3#3

Satisfied with your result???? Then, apply the filter in both videos.
* Submit a PDF file with Kalman class, the parameters and a brief explanation about your choice, and the new code with the loop in the video frames for tracking.
* Submit both videos with Kalman filter.
* Compare the results with and without filter, in the ramp and bouncing ball problems, and write your conclusions. Don't forget to discuss the filter parameters.







<font face="Verdana" size=6 color='#6495ED'>  Thank you for actively engaging in the Kalman Filter classes! We trust that you found the content enjoyable and enriching.

<center><img src='https://drive.google.com/uc?export=view&id=1fV0ruX4TCznLnNw-3ZV390p2JyU_-llu' width="600"></center>

Do you have any suggestions to enhance our classes? Feel free to share your thoughts by sending us a message; we would greatly appreciate and value your input.